# Chlorophyll-a measurement method uncertainty

## Preamble

### 1.  What the literature says


| Source  | Key quantitative      | Context  |
| ------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **O’Reilly et al., 2019** | • Fluorometric (FChl-a) random error exceeds HPLC (HChl-a) by **≈ 0.10–0.15 dex** when *Chl-a < 0.10 mg m⁻³*.<br>• Differences shrink above that threshold; means are close to 1:1.                                                                                                                                                                             | Global match-ups used to build **OC6** algorithm. Low-pigment oligotrophic focus.                                                                          |
| **Neeley et al., 2025**   | • Across 1 731 paired samples (coastal rivers + open ocean) the overall FChl-a vs HChl-a regression slope is **0.983** (≈ 1:1) and **R² = 0.996**.<br>• Bias small (0-2 %); *extra* FChl-a noise appears only when **Chl-a ≥ 3 mg m⁻³** in productive rivers.<br>• Violin plots show FChl-a σ comparable to (sometimes smaller than) HChl-a in many campaigns.  | Latest multi-cruise assessment aimed at PACE & Sentinel-2 validation; includes lab-diluted HPLC extracts measured on fluorometers (removes sampling bias). |


Dex = base-10 logarithmic unit (0.10 dex ≈ 26 % scatter; 0.30 dex ≈ factor 2).


### 2. What Model 6 learned from the NOMAD data

| Posterior summary |Interpretation |
| --------------- | ------------ |
| `γ_chl_type` (effect of fluorometry on log-σ) = **–0.109 ± 0.052 (94 % HDI = \[–0.207, –0.013])** | With our sample mix (mostly > 0.1 mg m⁻³, many coastal / riverine sites) **fluorometry appears \*less\* noisy than HPLC**. |

Why the sign flips relative to O’Reilly:

* Pigment range – few oligotrophic (< 0.1 mg m⁻³) points in our set.
* HPLC heterogeneity – multiple labs / storage protocols inflate HPLC σ (Neeley et al. discuss freezer losses up to 50 %).
* Indexing checked – fluorometry truly coded as 0.

### 3. Questions to be answered

| Policy / counter-factual                                | `pm.do()` action (Model 6c with `regime` indicator) | What insight it gives                                                                  |
| ------------------------------------------------------- | --------------------------------------------------- | -------------------------------------------------------------------------------------- |
| **If every sample were analysed by HPLC**               | `pm.do({"chl_type_idx": np.ones})`                  | Baseline “gold-standard” predictive σ.                                                 |
| **If every sample were fluorometric**                   | `pm.do({"chl_type_idx": np.zeros})`                 | Shows whether σ really drops (our posterior) or grows (literature).                    |
| **Impose O’Reilly extra noise (+0.12 dex) on fluo**     | `pm.do({"γ_chl_type": 0.12})`                       | How much predictive skill would deteriorate if paper’s penalty held here.              |
| **All data forced into low-Chl regime (< 0.10 mg m⁻³)** | `pm.do({"regime": np.ones})`                        | Directly tests the threshold hypothesis: does fluo variance surge only in that regime? |
